# Weyl 群扩张群的泛性质 

In [3]:
pwd

'/home/zhwang'

In [4]:
cd Documents/jupyter/0-items/WeylGroupExtension/

/home/zhwang/Documents/jupyter/0-items/WeylGroupExtension


In [5]:
load("weyl_group_extension.sage")

目前已有函数
1. ExpOfNilpotentMat(mat, max_depth=36, base_ring=ZZ) # 返回幂零阵的指数矩阵
2. Thetas(s,n) 返回生成元集合 \{\theta_i\}_{i=1}^n
3. OrderMatrixOfGens(gens) 返回生成元的群阶矩阵
4. GroupByOrderMathix(order_mat) 群阶矩阵作为泛性质，生成群
5. GroupTreeOfMaxDepth(G,depth) 生成群树（BFS）
6. UniversalPropertyOfGroup(G,max_depth=-1) 求群生成元的泛性质
7. relations2elements(relations,gens) 生成元通过 gens 转为群的子集

## 知识背景

### 极小泛性质

1. 上边算法求生成元泛性质，给出的是相对粗糙的刻画；
2. 一般地，很难有直接构造的算法，求“最精细”的刻画；
3. 难处可以归结为：对任意自由群商群，判断表示式是否等于1
4. 一些特殊情形，比如 Coxeter 群，Humphreys 书给出了判断等于1的充要方法；
5. 但就一般情形来说，这很难做到的，以至于专业研究群论的工具 GAP 有时都判断不了
6. 然而，对于有限群，我们可以有间接方法给出极小泛性质：先粗刻画，再逐步细化

### 函数说明及相关性质

1. 函数说明：
   - GroupTreeOfMaxDepth(G,depth) 群的 0-depth 层
   - UniversalPropertyOfGroup(G,max_depth=-1) 返回值有三个（默认生成整树）：
      - tree 群 G 的 0-max_depth 层
      - tree_str 群 G 的 0-max_depth 层，元素用 reduced expression  
      - relations 字典，**能够生成 0-max_depth 层，且键值 <= max_depth 的泛性质**
2. 泛性质等价
   - relations 给出的是粗刻画，需进一步加细
   - relations 可以将 0-max_depth 所有的 expression 简约化
   - 反之，如果 0-max_depth 所有的 expression 简约化已知，可导出 relations
   - 等价关系：若 relaitons1 和 relations2 的键值均在 0-max_depth 层，且将 0-max_depth 层的所有 expression 简约化，则二者等价
   - 等价的两个泛性质关系可相互生成
3. 基于上边的讨论，从泛性质的粗刻画出发，逐步细化

### Weyl 群及其扩张群的相关结论

观察前几阶的测试结论，易得性质猜想:
1. 确定 Weyl 群的泛性质，（至少）需要获取前 n 层：
   - A,D,E 型：前 3 层
   - B,C,F 型：前 4 层
   - G 型：前 6 层
2. 确定 Weyl 扩张群的泛性质，（至少）需要获取前 n 层：
   - A,D,E 型：前 4 层
   - B,C,F 型：前 4 层
   - G 型：前 6 层
3. 此外，二者泛性质可以只用包含两元素的情形

## 函数测试

In [8]:
a = {1:2,3:4}
b = [(key,a[key]) for key in a]

In [9]:
b

[(1, 2), (3, 4)]

In [ ]:
def MinimalUniversalProperty(G,max_depth=4):
    """
    1. 求群的极小泛性质
    2. max_depth 根据规律拟定（启发式）
    """
    tree_str,relations = UniversalPropertyOfGroup(G,max_depth)[1:]
    layer_len = [len(i) for i in tree_str] # 记录各层长度数据
    relations = [(key,relations[key]) for key in relations] # 关系转列表

In [12]:
def UniversalPropertyOfGroup(G,max_depth=-1):
    """
    1. 求群生成元的泛性质
    2. G 的生成元不能存在相等，或者等于1
    3. max_depth 获取层数，默认遍历整树
    """
    if max_depth == 0:
        return [[G.one()]],[[""]],[]
    # 检验输入
    gens = list(G.gens()) # 生成元
    S = [G.one()] + gens # 已生成群元素
    assert len(S)==len(set(S)),"G 的生成元不能存在相等，或者等于1"
    
    # 群树：元素形式和字符串形式
    tree = [[G.one()],gens] 
    tree_str = [[""],["%d"%i for i in range(len(gens))]]
    # 生成关系
    relations = [[],[]]
    while max_depth-1:
        max_depth -= 1
        new_layer = []
        new_layer_str = []
        for s,a in zip(tree_str[-1],tree[-1]):
            for si,b in enumerate(gens):
                c = a * b
                c_str = s + "-%d"%si
                if c in new_layer: # 化简到同一层
                    if not any(s in c_str for s in relations[0]):
                        relations[0].append(c_str)
                        relations[1].append(new_layer_str[new_layer.index(c)])
                    continue
                for i,layer in enumerate(tree):
                    if c in layer: # 化简到前边层
                        if not any(s in c_str for s in relations[0]):
                            relations[0].append(c_str)
                            relations[1].append(tree_str[i][layer.index(c)])
                        break
                else:
                    new_layer.append(c)
                    new_layer_str.append(c_str)
        if len(new_layer):
            tree.append(new_layer)
            tree_str.append(new_layer_str)
        else:
            break
    relations = [(a,b) for a,b in zip(*relations)]
    return tree,tree_str,relations